In [1]:
import numpy as np
import pandas as pd
from IPython.display import display as display_dataframe
import pickle
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential, save_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import optimizers
import keras
# from keras.applications.xception import Xception
# from keras.applications.resnet50 import ResNet50
# from keras.applications.vgg16 import VGG16
# from keras.applications.vgg19 import VGG19
# from keras.applications.inception_v3 import InceptionV3
# from keras.applications.mobilenetv2 import MobileNetV2
# from keras.applications.densenet import DenseNet121
import cv2
import os
import seaborn as sns
# from keras.preprocessing.image import load_img, img_to_array


import warnings
warnings.filterwarnings('ignore')

def display_image(image, title=''):
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Using TensorFlow backend.


# load data

In [2]:
#vgg can't take input of 28x28 so we shall resize all our images
pickle_folder = "pickle files"
data = pickle.load(open('data.pickle', 'rb'))
print("Data of format:")
for ch in data:
    print("data_length(" + ch + ") =", len(data[ch]))

#no. of classes
classes = len(data.keys())

Data of format:
data_length(ण) = 2000
data_length(फ) = 2000
data_length(ज्ञ) = 2000
data_length(द) = 2000
data_length(ल) = 2000
data_length(ट) = 2000
data_length(र) = 2000
data_length(ब) = 2000
data_length(व) = 2000
data_length(ङ) = 2000
data_length(त्र) = 2000
data_length(न) = 2000
data_length(ष) = 2000
data_length(ड) = 2000
data_length(ज) = 2000
data_length(श) = 2000
data_length(ठ) = 2000
data_length(त) = 2000
data_length(थ) = 2000
data_length(झ) = 2000
data_length(घ) = 2000
data_length(ख) = 2000
data_length(ञ) = 2000
data_length(क्ष) = 2000
data_length(ढ) = 2000
data_length(भ) = 2000
data_length(छ) = 2000
data_length(ध) = 2000
data_length(ह) = 2000
data_length(य) = 2000
data_length(च) = 2000
data_length(क) = 2000
data_length(स) = 2000
data_length(ग) = 2000
data_length(म) = 2000
data_length(प) = 2000


# Rearranging data in input-output format

In [3]:
file = "class_numerals_{}.pickle".format(classes)
if file in os.listdir(pickle_folder):
    class_numerals = pickle.load(open(os.path.join(pickle_folder, file), 'rb'))
    print("Unpickled")
else:
    class_numerals = {i:list(data.keys())[i] for i in range(classes)}
    pickle.dump(class_numerals, open(os.path.join(pickle_folder, file), 'wb'))
    print("Error. Incompatible. Pickled new file:", file)
def character_to_encoding(char, classes=classes):
    for i in class_numerals:
        if class_numerals[i] == char:
            return np_utils.to_categorical(i, classes)

def encoding_to_character(encoding):
    return class_numerals[encoding.argmax()]

Unpickled


# CNN

In [4]:
import tensorflow as tf
from keras import backend as k
 
###################################
# TensorFlow wizardry
config = tf.ConfigProto()
 
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True
 
# Only allow a total of half the GPU memory to be allocated
# config.gpu_options.per_process_gpu_memory_fraction = 0.5
 
# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))


print("GPUs:", k.tensorflow_backend._get_available_gpus())

# print("input shape:", input_shape)
print("classes:", classes)

def get_model_memory_usage(batch_size, model):
    import numpy as np
    from keras import backend as K

    shapes_mem_count = 0
    for l in model.layers:
        single_layer_mem = 1
        for s in l.output_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = np.sum([K.count_params(p) for p in set(model.trainable_weights)])
    non_trainable_count = np.sum([K.count_params(p) for p in set(model.non_trainable_weights)])

    total_memory = 4.0*batch_size*(shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3)
    return gbytes

GPUs: ['/job:localhost/replica:0/task:0/device:GPU:0']
classes: 36


In [6]:
%%time

batch_size = 128
num_classes = classes
epochs = 500
k_r = 0.01
b_r = 0.01
dropout = 0.1

early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=0, mode='auto')
input_shape=(32, 32, 1)
results = pd.DataFrame(columns=['Label_count']+["Label#" + str(i) for i in range(1, classes+1)]+["Test_acc", "Overall_acc"])
display_dataframe(results)

for number_of_characters_at_once in range(36, 37):
    random_iterations = 1
    for iteration in range(random_iterations):
        #selecting labels
        labels = list(data.keys())
        np.random.shuffle(labels)
        labels = labels[:number_of_characters_at_once]
        print("\n", iteration, labels)
        
        #preparing relevant data
        X = []
        y = []
        for ch in labels:
            X += data[ch]
            y += [ch] * len(data[ch])

        #resizing input image and preprocessin
        X = np.array(list(map(lambda x: cv2.resize(x/255, (input_shape[0], input_shape[1])).reshape((input_shape[0], input_shape[1], 1)), X)))
        y_ = np.array(list(map(lambda x: character_to_encoding(x, classes), y)))
        print(X.shape, y_.shape)

        #train test split
        train_ratio = 0.8
        X_train, X_test, y_train, y_test = train_test_split(X, y_, train_size=0.8, random_state=20, shuffle=True, stratify=y)
        print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
        
        
        #the model itself
        model = Sequential()
        
        model.add(Conv2D(6, kernel_size=(5, 5),
                         activation='relu',
        #                   kernel_initializer='he_normal',
        #                   kernel_regularizer=keras.regularizers.l2(k_r),
        #                   bias_regularizer=keras.regularizers.l2(b_r),
                         input_shape=input_shape))
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(dropout))
        model.add(Conv2D(12, kernel_size=(5, 5),
                         activation='relu',
        #                   kernel_initializer='he_normal',
        #                   kernel_regularizer=keras.regularizers.l2(k_r),
        #                   bias_regularizer=keras.regularizers.l2(b_r)
                        ))
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(dropout))
        model.add(Conv2D(120, kernel_size=(5, 5),
                         activation='relu',
        #                   kernel_initializer='he_normal',
        #                   kernel_regularizer=keras.regularizers.l2(k_r),
        #                   bias_regularizer=keras.regularizers.l2(b_r)
                        ))

        # NN
        model.add(Flatten())
        #hidden layer 1
        model.add(Dense(512, activation='relu', kernel_initializer='he_normal',
                        kernel_regularizer=keras.regularizers.l2(k_r)))
        model.add(Dropout(dropout))
        #output laye1
        model.add(Dense(num_classes, activation='softmax'))

        model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
        print(model.summary())
        print("\n############################")
        print("## Memory usage:", get_model_memory_usage(batch_size=batch_size, model=model), "GB ##")
        print("############################\n")

        #adding custom weights
        model.set_weights(keras.models.load_model(os.path.join(pickle_folder, "lenet_relu_3_conv_layer.h5")).get_weights())
        
        #run model
        model.fit(X_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  verbose=1,
                  validation_split=0.15,
                callbacks = [early_stopping_callback])

        #save model
        filename = "Lenet_" + "_".join(labels) + '.h5'
        try:
            save_model(model, os.path.join(pickle_folder, filename))
        except Exception as e:
            os.remove(os.path.join(pickle_folder, filename))
            save_model(model, os.path.join(pickle_folder, filename))
            print(e)
        
        #update results
        results.loc[number_of_characters_at_once*random_iterations + iteration] = [len(labels)] + labels + ['']*(classes-len(labels)) + [model.evaluate(X_test, y_test, verbose=0)[1], model.evaluate(X, y_, verbose=0)[1]]        
        display_dataframe(results.tail(5))
        results.to_csv(os.path.join(pickle_folder,'results_'+str(iteration) + '.csv'))

,Label_count,Label#1,Label#2,Label#3,Label#4,Label#5,Label#6,Label#7,Label#8,Label#9,...,Label#29,Label#30,Label#31,Label#32,Label#33,Label#34,Label#35,Label#36,Test_acc,Overall_acc



 0 ['ब', 'य', 'ञ', 'त', 'क्ष', 'फ', 'ठ', 'स', 'छ', 'ज्ञ', 'च', 'ण', 'द', 'र', 'ट', 'म', 'प', 'क', 'ङ', 'भ', 'न', 'ल', 'ह', 'त्र', 'ज', 'ख', 'झ', 'थ', 'घ', 'श', 'ष', 'ग', 'व', 'ढ', 'ड', 'ध']
(72000, 32, 32, 1) (72000, 36)
(57600, 32, 32, 1) (57600, 36) (14400, 32, 32, 1) (14400, 36)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_61 (Conv2D)           (None, 28, 28, 6)         156       
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 14, 14, 6)         0         
_________________________________________________________________
dropout_61 (Dropout)         (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_62 (Conv2D)           (None, 10, 10, 12)        1812      
_________________________________________________________________
max_pooling2d_42 (MaxPooling (None, 5, 5, 12)          0

,Label_count,Label#1,Label#2,Label#3,Label#4,Label#5,Label#6,Label#7,Label#8,Label#9,...,Label#29,Label#30,Label#31,Label#32,Label#33,Label#34,Label#35,Label#36,Test_acc,Overall_acc
36,36,ब,य,ञ,त,क्ष,फ,ठ,स,छ,...,घ,श,ष,ग,व,ढ,ड,ध,0.952569,0.974889


CPU times: user 1min 23s, sys: 9.38 s, total: 1min 32s
Wall time: 1min 32s
